In [1]:
using DataFrames
using GLM
using Requests

# Load data

In [9]:
resp = get("https://raw.githubusercontent.com/matthijscox/many-models/master/many_models_data.csv")
df = readtable(IOBuffer(resp.data))
df[end-2:end,:]

,num_group,cat_group,A,B,Z
1,2000,Y,-0.135,-0.0095,-3232.24354363029
2,2000,Y,0.105,-0.0475,-1394.27444233176
3,2000,Y,0.045,-0.0475,-1857.96462068288


In [3]:
# Experimenting with selecting elements, works great!
df[end-10:end,[:num_group,:cat_group,:Z]]
idx = (df[:num_group].==1) .& (df[:cat_group].=="X")
size(df[idx,:]) 

(80, 5)

# Function definitions

In [4]:
# Using the GLM package
function calc_poly_residuals(df)
    # Q: how to input quadratic terms, such as A^2 and A^2*B?
    OLS = fit(LinearModel,@formula(Z ~ A*B), df)
    resid = df[:Z]-predict(OLS)
end

df2 = df[idx,:]
df2[:resid] = calc_poly_residuals(df2)
df2[1:3,:]

,num_group,cat_group,A,B,Z,resid
1,1,X,0.015,-0.0095,3868.33898544624,7.81984931630268
2,1,X,-0.105,0.1045,5307.37911587364,3.17880082490683
3,1,X,-0.135,-0.0475,3384.50800319023,8.422838793980645


In [5]:
# 2D polynomial model matrix
function model_matrix_poly(x,y,poly_order=3)
    ndat = length(x)
    npoly = (poly_order+1)*(poly_order+2)
    M = ones(ndat,npoly/2)
    for pow = 1:poly_order
        for pow_y = 0:pow
            pow_x = pow-pow_y
            M[:,round(Int,pow*(pow+1)/2+pow_y+1)] = (x.^pow_x .* y.^pow_y)
        end
    end
    return M
end

# test it:
M = model_matrix_poly(df2[:A],df2[:B],3)


80×10 Array{Float64,2}:
 1.0   0.015  -0.0095  0.000225  -0.0001425  …   1.35375e-6   -8.57375e-7 
 1.0  -0.105   0.1045  0.011025  -0.0109725     -0.00114663    0.00114117 
 1.0  -0.135  -0.0475  0.018225   0.0064125     -0.000304594  -0.000107172
 1.0   0.045   0.1425  0.002025   0.0064125      0.000913781   0.00289364 
 1.0  -0.015  -0.1425  0.000225   0.0021375     -0.000304594  -0.00289364 
 1.0   0.135   0.0475  0.018225   0.0064125  …   0.000304594   0.000107172
 1.0   0.105  -0.1045  0.011025  -0.0109725      0.00114663   -0.00114117 
 1.0  -0.015   0.0855  0.000225  -0.0012825     -0.000109654   0.000625026
 1.0  -0.075   0.0095  0.005625  -0.0007125     -6.76875e-6    8.57375e-7 
 1.0  -0.045  -0.0665  0.002025   0.0029925     -0.000199001  -0.00029408 
 1.0   0.105  -0.0285  0.011025  -0.0029925  …   8.52863e-5   -2.31491e-5 
 1.0   0.075   0.0855  0.005625   0.0064125      0.000548269   0.000625026
 1.0  -0.045   0.1425  0.002025  -0.0064125     -0.000913781   0.00289364 
 

In [6]:
# using model matrices directly
function calc_poly_residuals_with_matrices(df)        
    # Using a model matrix and mldivide
    M = model_matrix_poly(df[:A],df[:B])
    y = convert(Array,df[:Z])
    c = M \ y
    resid = y-M*c
end

calc_poly_residuals_with_matrices(df[idx,:])


80-element Array{Float64,1}:
 -0.423496 
  0.418609 
 -0.226803 
 -0.944066 
 -0.84604  
  0.456776 
  0.246115 
 -0.28419  
 -0.288782 
  0.454852 
  0.337804 
 -0.512187 
 -0.230121 
  ⋮        
  0.609347 
  0.658001 
 -0.383448 
  0.0302925
  0.749151 
  0.0227712
 -0.488238 
 -0.215116 
 -0.540411 
 -0.205108 
 -0.231882 
  0.466321 

# Julia many models approach

In [10]:
# quite impressive timing for dataframes (0.7 seconds using matrix approach, 1.5s using GLM)

@time resid = by(df,[:num_group,:cat_group],df -> DataFrame(resid = calc_poly_residuals(df)))[:resid]
@time resid = by(df,[:num_group,:cat_group],df -> DataFrame(resid = calc_poly_residuals_with_matrices(df)))[:resid]

# root-mean-square compares well to Matlab result
mat_diff = sqrt(sum(resid.^2)/size(resid,1))-0.878223092545709
print("Julia difference to reference: $mat_diff")

# reference:
# https://juliadata.github.io/DataFrames.jl/stable/man/split_apply_combine/

  1.441925 seconds (6.63 M allocations: 330.713 MiB, 6.72% gc time)
  0.598673 seconds (6.91 M allocations: 540.053 MiB, 18.60% gc time)
Julia difference to reference: 6.106226635438361e-15